## 참고: heapq 라이브러리 활용을 통해 우선순위 큐 사용하기¶

- heapq.heappush를 통해 데이터를 heap 형태로 넣을 수 있음 (0번 인덱스를 우선순위로 인지함)
- heapq.heapify() 함수를 통해 리스트 데이터를 heap 형태로 한 번에 변환할 수 있음 (0번 인덱스를 우선순위로 인지함)

In [9]:
import heapq

graph_data = [[2, 'A'], [5, 'B'], [1, 'C'], [7, 'D']]


heapq.heapify(graph_data)
print(graph_data)
for index in range(len(graph_data)):
    print (heapq.heappop(graph_data))

print (graph_data)


[[1, 'C'], [5, 'B'], [2, 'A'], [7, 'D']]
[1, 'C']
[2, 'A']
[5, 'B']
[7, 'D']
[]


## 참고: collections 라이브러리의 defaultdict 함수 활용하기

- defaultdict 함수를 사용해서, key에 대한 value를 지정하지 않았을 시, 빈 리스트로 초기화하기


In [17]:
from collections import defaultdict

list_dict = defaultdict(list)
print (list_dict['sdf'])


[]


## 프림 알고리즘 파이썬 코드

1. 모든 간선 정보를 저장 (adjacent_edges)
2. 임의의 정점을 선택, '연결된 노드 집합(connected_nodes)'에 삽입
3. 선택된 정점에 연결된 간선들을 간선 리스트(candidate_edge_list)에 삽입
4. 간선 리스트(candidate_edge_list)에서 최소 가중치를 가지는 간선부터 추출해서,
    - 해당 간선에 연결된 인접 정점이 '연결된 노드 집합'에 이미 들어 있다면, 스킵함. 
    (cycle 발생을 막기 위함)
    - 해당 간선에 연결된 인접 정점이 '연결된 노드 집합'에 들어 있지 않으면, 해당 간선을 선택하고, 해당 간선 정보를 '최소 신장 트리(mst)'에 삽입
        - 해당 간선에 연결된 인접 정점의 간선들 중, '연결된 노드 집합(connected_nodes)' 에 없는 노드와 연결된 간선들만 간선 리스트(candidate_edge_list) 에 삽입
        - 어차피 스킵될 간선을 간선 리스트(candidate_edge_list)에 넣지 않으므로 해서, 간선 리스트(candidate_edge_list)에서 최소 가중치를 가지는 간선부터 추출하기 위한 자료구조 유지를 위한 effort를 줄일 수 있음 (예, 최소힙 구조 사용)
5. 선택된 간선은 간선 리스트에서 제거
6. 간선 리스트에 더 이상의 간선이 없을 때까지 3-4번을 반복

In [18]:
myedges = [
    (7, 'A', 'B'), (5, 'A', 'D'),
    (8, 'B', 'C'), (9, 'B', 'D'), (7, 'B', 'E'),
    (5, 'C', 'E'),
    (7, 'D', 'E'), (6, 'D', 'F'),
    (8, 'E', 'F'), (9, 'E', 'G'),
    (11, 'F', 'G')
]

from collections import defaultdict
from heapq import *

def prim(start_node, edges):
    mst = list()
    adjacent_edges = defaultdict(list)
    for weight, n1, n2 in edges:
        adjacent_edges[n1].append((weight, n1, n2))
        adjacent_edges[n2].append((weight, n2, n1))
    print(adjacent_edges)

    # 시작 정점을 '연결된 노드 집합(connected_nodes)'에 삽입
    connected_nodes = set(start_node)
    
    # 선택된 정점에 연결된 간선들을 간선 리스트(candidate_edge_list)에 삽입
    candidate_edge_list = adjacent_edges[start_node]
    
    #데이터를 heap 형태로 한 번에 변환
    heapify(candidate_edge_list)
    
    while candidate_edge_list:
        weight, n1, n2 = heappop(candidate_edge_list)
        
        # 해당 간선에 연결된 인접 정점이 '연결된 노드 집합'에 들어 있지 않으면,
        # 해당 간선 정보를 '최소 신장 트리(mst)'에 삽입
        if n2 not in connected_nodes:
            connected_nodes.add(n2)
            mst.append((weight, n1, n2))
            
            for edge in adjacent_edges[n2]:
                if edge[2] not in connected_nodes:
                    heappush(candidate_edge_list, edge)

    return mst


prim ('A', myedges)


defaultdict(<class 'list'>, {'A': [(7, 'A', 'B'), (5, 'A', 'D')], 'B': [(7, 'B', 'A'), (8, 'B', 'C'), (9, 'B', 'D'), (7, 'B', 'E')], 'D': [(5, 'D', 'A'), (9, 'D', 'B'), (7, 'D', 'E'), (6, 'D', 'F')], 'C': [(8, 'C', 'B'), (5, 'C', 'E')], 'E': [(7, 'E', 'B'), (5, 'E', 'C'), (7, 'E', 'D'), (8, 'E', 'F'), (9, 'E', 'G')], 'F': [(6, 'F', 'D'), (8, 'F', 'E'), (11, 'F', 'G')], 'G': [(9, 'G', 'E'), (11, 'G', 'F')]})


[(5, 'A', 'D'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (5, 'E', 'C'),
 (9, 'E', 'G')]

## 참고: 개선된 프림 알고리즘

- 간선이 아닌 노드를 중심으로 우선순위 큐를 적용하는 방식
    - 초기화 - **정점:key**  구조를 만들어놓고, 특정 정점의 key값은 0, 이외의 정점들의 key값은 무한대로 놓음. 모든  **정점:key**  값은 우선순위 큐에 넣음
    - 가장 key값이 적은 **정점:key** 를 추출한 후(pop 하므로 해당 정점:key 정보는 우선순위 큐에서 삭제됨), (extract min 로직이라고 부름)
    - 해당 정점의 인접한 정점들에 대해 key 값과 연결된 가중치 값을 비교하여 key값이 작으면 해당 **정점:key** 값을 갱신
        - **정점:key** 값 갱신시, 우선순위 큐는 최소 key값을 가지는 **정점:key** 를 루트노드로 올려놓도록 재구성함 (decrease key 로직이라고 부름)

In [35]:
from heapdict import heapdict

def prim(graph, start):
    mst, keys, pi, total_weight = list(), heapdict(), dict(), 0
    for node in graph.keys():
        keys[node] = float('inf')
        pi[node] = None
    keys[start], pi[start] = 0, start
    
    for a, b in keys.items():
        print(a, b)
    print("---")
    for a, b in pi.items():
        print(a, b)
        
    while keys:
        current_node, current_key = keys.popitem()
        print(current_node, current_key)
        mst.append([pi[current_node], current_node, current_key])
        print(mst)
        total_weight += current_key
        
        # mygraph의 key 값들 루프
        for adjacent, weight in mygraph[current_node].items():
            if adjacent in keys and weight < keys[adjacent]:
                keys[adjacent] = weight
                pi[adjacent] = current_node
                print("pi.items()", pi.items())
    return mst, total_weight

In [36]:
mygraph = {
    'A': {'B': 7, 'D': 5},
    'B': {'A': 7, 'D': 9, 'C': 8, 'E': 7},
    'C': {'B': 8, 'E': 5},
    'D': {'A': 5, 'B': 9, 'E': 7, 'F': 6},
    'E': {'B': 7, 'C': 5, 'D': 7, 'F': 8, 'G': 9},
    'F': {'D': 6, 'E': 8, 'G': 11},
    'G': {'E': 9, 'F': 11}    
}
mst, total_weight = prim(mygraph, 'A')
print ('MST:', mst)
print ('Total Weight:', total_weight)

B inf
C inf
D inf
E inf
F inf
G inf
A 0
---
A A
B None
C None
D None
E None
F None
G None
A 0
[['A', 'A', 0]]
pi.items() dict_items([('A', 'A'), ('B', 'A'), ('C', None), ('D', None), ('E', None), ('F', None), ('G', None)])
pi.items() dict_items([('A', 'A'), ('B', 'A'), ('C', None), ('D', 'A'), ('E', None), ('F', None), ('G', None)])
D 5
[['A', 'A', 0], ['A', 'D', 5]]
pi.items() dict_items([('A', 'A'), ('B', 'A'), ('C', None), ('D', 'A'), ('E', 'D'), ('F', None), ('G', None)])
pi.items() dict_items([('A', 'A'), ('B', 'A'), ('C', None), ('D', 'A'), ('E', 'D'), ('F', 'D'), ('G', None)])
F 6
[['A', 'A', 0], ['A', 'D', 5], ['D', 'F', 6]]
pi.items() dict_items([('A', 'A'), ('B', 'A'), ('C', None), ('D', 'A'), ('E', 'D'), ('F', 'D'), ('G', 'F')])
B 7
[['A', 'A', 0], ['A', 'D', 5], ['D', 'F', 6], ['A', 'B', 7]]
pi.items() dict_items([('A', 'A'), ('B', 'A'), ('C', 'B'), ('D', 'A'), ('E', 'D'), ('F', 'D'), ('G', 'F')])
E 7
[['A', 'A', 0], ['A', 'D', 5], ['D', 'F', 6], ['A', 'B', 7], ['D', 'E', 7